Need an EP and CP  ts pattern and an EP and CP pr pattern for pattern correlation with models. 
Will use 20CR. Can use HadISST for nino index and EOFs. Need monthly temperature anomalies. 

In [1]:
import xarray as xr
import xesmf
import cf_xarray
import numpy as np
import sys
sys.path.append('/home/563/rl5183/PhD-research/Functions')
import functions as func

In [2]:
# Open 20CR ts data
ds = xr.open_dataset('/g/data/w40/rl5183/20CR/temp/monthly_tmp_all_members.nc').chunk('auto')
# Take ensemble mean
data = ds.TMP
data = data.mean(dim='member').chunk('auto')
# Regrid
sample_ds = xr.open_dataset('/g/data/w40/rl5183/CMIP6_20_E_temp_regressions')
bnds = ds.cf.add_bounds(['lat','lon'])
sample_bnds = sample_ds.cf.add_bounds(['lat','lon'])
regrid = xesmf.Regridder(bnds, sample_bnds, method='conservative_normed')
data_regrid = regrid(data).chunk(chunks={'time':-1, 'lat':60})
# Quadtratically detrend 
data_dt = xr.apply_ufunc(func.quad_detrend, data_regrid, 
                             input_core_dims=[['time']],
                             output_core_dims=[['time']],
                             vectorize=True,
                             dask='parallelized')

data_dt = data_dt.transpose(*data.dims).sel(time=slice('1900-01', '2014-12'))
# Save as netcdf
data_dt.to_netcdf('/g/data/w40/rl5183/20CR_tmp_ensemble_mean_quad_detrend_1.5x1.5.nc')

In [3]:
# Open 20CR pr data, regrid, and save as netcdf 
pr = xr.open_dataset('/g/data/w40/rl5183/20CR/precip/monthly_prate_all_members.nc').PRATE.chunk('auto')
pr = pr.mean(dim='member').chunk('auto')
pr_regrid = regrid(pr)
pr_regrid.to_netcdf('/g/data/w40/rl5183/20CR_pr_ensemble_mean_1.5x1.5.nc')

In [5]:
xr.open_dataarray('/g/data/w40/rl5183/20CR_tmp_ensemble_mean_quad_detrend_1.5x1.5.nc').chunk('auto')

<xarray.DataArray (time: 1380, lat: 120, lon: 240)>
dask.array<xarray-<this-array>, shape=(1380, 120, 240), dtype=float64, chunksize=(460, 120, 240), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1900-01-16T10:30:00 ... 2014-12-16T10:30:00
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25